In [20]:
import pandas as pd
import numpy as np
import re
import requests

In [48]:
dataset_path = '1000_cleaning/output/1000_downloaded_dataset.csv'
cleaned_dataset_path = '1000_cleaning/output/1000_cleaning.csv'
total_occurrences_by_division_path = '1000_cleaning/output/1000_total_occurrences_by_division.csv'

# Downloading the dataset

The dataset we are using comes from the [Central Statistics Office of Ireland](cso.ie)

The information we will be using from this dataset is:
* `Year`
* `County/Garda Station Division`
* `Type of offence`
* `Number of occurences`

Aswell as the index of a crime relative to its garda station called `Crime Index`

In [ ]:
url = 'https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/CJA07/CSV/1.0/'
r = requests.get(url, allow_redirects=True)
open(dataset_path, 'wb').write(r.content)

# Cleaning the dataset

We need to remove any columns that have useless data, aswell as rename some of the poorly named columns for better readability later on.

Every cell in crime index gets subtracted by `3` to bring it into a more usable index range. *0-12 instead of 3-15*

The final lambda changes the cells in `Garda Station` from `[Station ID] [Town], [County] Division` to `[County] Division`, since we will not be using the `[Town]` or `[Station ID]` 

In [39]:
dataset = pd.read_csv(dataset_path)
dataset = dataset.drop(columns=['STATISTIC', 'STATISTIC Label','UNIT','TLIST(A1)','C03037V03742'],axis=4)
dataset = dataset.rename(columns={'C02480V03003': 'Crime Index', 'VALUE': 'Occurrences'})
dataset['Crime Index'] = dataset['Crime Index'] - 3
dataset['Garda Station'] = dataset['Garda Station'].apply(lambda x: re.search('\s([^,]+$)',x).group(1))

In [40]:
dataset.head()

,Year,Garda Station,Crime Index,Type of Offence,Occurrences
0,2003,Limerick Division,0,"Attempts/threats to murder, assaults, harassme...",18
1,2003,Limerick Division,1,Dangerous or negligent acts,14
2,2003,Limerick Division,2,Kidnapping and related offences,0
3,2003,Limerick Division,3,"Robbery, extortion and hijacking offences",0
4,2003,Limerick Division,4,Burglary and related offences,27


In [ ]:
sums = dataset.groupby(by='Garda Station')['Occurrences'].sum()

In [29]:
sums.head()

Garda Station
Cavan/Monaghan Division    102052
Clare Division              78677
Cork City Division         264864
Cork North Division         89914
Cork West Division          63400
Name: Occurrences, dtype: int64

In [49]:
dataset.to_csv(cleaned_dataset_path)
sums.to_csv(total_occurrences_by_division_path)